In [61]:
import nibabel as nib
import imageio
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import torch.optim as optim
import torch.nn as nn
from tqdm.notebook import tqdm
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import torch
import segmentation_models_pytorch as smp
import albumentations as albu
from PIL import Image
import re
import subprocess
import pre_proc_functions_031624 as proc
from natsort import natsorted
import pandas as pd
import shutil
import inference_flex_functions as inf

In [3]:
#input_directory
images_dir = "/home/zimu/Desktop/Code/Skull_Stripe_dataset/zhao_data/img"
#Study Name
study_name = "zhao_res"
metrics_out = "zhao_res"


truth_folder = 'masks_swapped_val'

images_output_dir = study_name + "/raw_png"
raw_png_output = study_name + '/raw_png_output'
raw_npy_output = study_name + '/raw_npy_output'
volumn_out = study_name + "/volumn_out"
prob_out = study_name + "/prob_out"
final_out = study_name + '/final_out'

if not os.path.exists(images_output_dir):
    os.makedirs(images_output_dir)
    os.makedirs(os.path.join(images_output_dir,'sag'))
    os.makedirs(os.path.join(images_output_dir,'ax'))
    os.makedirs(os.path.join(images_output_dir,'cor'))
raw_png_sag = os.path.join(images_output_dir,'sag')
raw_png_ax = os.path.join(images_output_dir,'ax')
raw_png_cor = os.path.join(images_output_dir,'cor')

if not os.path.exists(raw_png_output):
    os.makedirs(raw_png_output)
    os.makedirs(os.path.join(raw_png_output,'sag'))
    os.makedirs(os.path.join(raw_png_output,'ax'))
    os.makedirs(os.path.join(raw_png_output,'cor'))

raw_png_sag_output = os.path.join(raw_png_output,'sag')
raw_png_ax_output = os.path.join(raw_png_output,'ax')
raw_png_cor_output = os.path.join(raw_png_output,'cor')

if not os.path.exists(raw_npy_output):
    os.makedirs(raw_npy_output)
    os.makedirs(os.path.join(raw_npy_output,'sag'))
    os.makedirs(os.path.join(raw_npy_output,'ax'))
    os.makedirs(os.path.join(raw_npy_output,'cor'))

raw_npy_sag_output = os.path.join(raw_npy_output,'sag')
raw_npy_ax_output = os.path.join(raw_npy_output,'ax')
raw_npy_cor_output = os.path.join(raw_npy_output,'cor')

if not os.path.exists(volumn_out):
    os.makedirs(volumn_out)
    os.makedirs(os.path.join(volumn_out,'sag'))
    os.makedirs(os.path.join(volumn_out,'ax'))
    os.makedirs(os.path.join(volumn_out,'cor'))

volumn_out_sag = os.path.join(volumn_out,'sag')
volumn_out_ax = os.path.join(volumn_out,'ax')
volumn_out_cor = os.path.join(volumn_out,'cor')

if not os.path.exists(prob_out):
    os.makedirs(prob_out)
    os.makedirs(os.path.join(prob_out,'sag'))
    os.makedirs(os.path.join(prob_out,'ax'))
    os.makedirs(os.path.join(prob_out,'cor'))

prob_out_sag = os.path.join(prob_out,'sag')
prob_out_ax = os.path.join(prob_out,'ax')
prob_out_cor = os.path.join(prob_out,'cor')

In [4]:
proc.proc_img_masks(img_dir=images_dir, out_dir=images_output_dir,img_fixed = "_mc_restore", mask_fixed = '-mask')

Pig_28
Processed zhao_res/raw_png/./sag/Pig_28_slice000.png
Processed zhao_res/raw_png/./sag/Pig_28_slice001.png
Processed zhao_res/raw_png/./sag/Pig_28_slice002.png
Processed zhao_res/raw_png/./sag/Pig_28_slice003.png
Processed zhao_res/raw_png/./sag/Pig_28_slice004.png
Processed zhao_res/raw_png/./sag/Pig_28_slice005.png
Processed zhao_res/raw_png/./sag/Pig_28_slice006.png
Processed zhao_res/raw_png/./sag/Pig_28_slice007.png
Processed zhao_res/raw_png/./sag/Pig_28_slice008.png
Processed zhao_res/raw_png/./sag/Pig_28_slice009.png
Processed zhao_res/raw_png/./sag/Pig_28_slice010.png
Processed zhao_res/raw_png/./sag/Pig_28_slice011.png
Processed zhao_res/raw_png/./sag/Pig_28_slice012.png
Processed zhao_res/raw_png/./sag/Pig_28_slice013.png
Processed zhao_res/raw_png/./sag/Pig_28_slice014.png
Processed zhao_res/raw_png/./sag/Pig_28_slice015.png
Processed zhao_res/raw_png/./sag/Pig_28_slice016.png
Processed zhao_res/raw_png/./sag/Pig_28_slice017.png
Processed zhao_res/raw_png/./sag/Pig_28

In [6]:
files_list_sag = inf.get_files_starting_with(raw_png_sag, "Pig")
print(sorted(files_list_sag))
files_list_sag = sorted(files_list_sag)

files_list_ax = inf.get_files_starting_with(raw_png_ax, "Pig")
print(sorted(files_list_ax))
files_list_ax = sorted(files_list_ax)

files_list_cor = inf.get_files_starting_with(raw_png_cor, "Pig")
print(sorted(files_list_cor))
files_list_cor = sorted(files_list_cor)

['zhao_res/raw_png/sag/Pig_25_slice000.png', 'zhao_res/raw_png/sag/Pig_25_slice001.png', 'zhao_res/raw_png/sag/Pig_25_slice002.png', 'zhao_res/raw_png/sag/Pig_25_slice003.png', 'zhao_res/raw_png/sag/Pig_25_slice004.png', 'zhao_res/raw_png/sag/Pig_25_slice005.png', 'zhao_res/raw_png/sag/Pig_25_slice006.png', 'zhao_res/raw_png/sag/Pig_25_slice007.png', 'zhao_res/raw_png/sag/Pig_25_slice008.png', 'zhao_res/raw_png/sag/Pig_25_slice009.png', 'zhao_res/raw_png/sag/Pig_25_slice010.png', 'zhao_res/raw_png/sag/Pig_25_slice011.png', 'zhao_res/raw_png/sag/Pig_25_slice012.png', 'zhao_res/raw_png/sag/Pig_25_slice013.png', 'zhao_res/raw_png/sag/Pig_25_slice014.png', 'zhao_res/raw_png/sag/Pig_25_slice015.png', 'zhao_res/raw_png/sag/Pig_25_slice016.png', 'zhao_res/raw_png/sag/Pig_25_slice017.png', 'zhao_res/raw_png/sag/Pig_25_slice018.png', 'zhao_res/raw_png/sag/Pig_25_slice019.png', 'zhao_res/raw_png/sag/Pig_25_slice020.png', 'zhao_res/raw_png/sag/Pig_25_slice021.png', 'zhao_res/raw_png/sag/Pig_25_sl

In [7]:
model_sag = torch.load('/home/zimu/Desktop/Code/Transfer_learning/zhao_checkpoints/29_sag.pth')
model_cor = torch.load('/home/zimu/Desktop/Code/Transfer_learning/zhao_checkpoints/29_cor.pth')
model_ax = torch.load('/home/zimu/Desktop/Code/Transfer_learning/zhao_checkpoints/29_ax.pth')
preprocessing_fn = smp.encoders.get_preprocessing_fn("efficientnet-b3", "imagenet")
sag_dataset = inf.Dataset(
    images_dir = raw_png_sag, 
    preprocessing=inf.get_preprocessing(preprocessing_fn),
    classes=['brain'],
)

cor_dataset = inf.Dataset(
    images_dir = raw_png_cor, 
    preprocessing=inf.get_preprocessing(preprocessing_fn),
    classes=['brain'],
)

ax_dataset = inf.Dataset(
    images_dir = raw_png_ax, 
    preprocessing=inf.get_preprocessing(preprocessing_fn),
    classes=['brain'],
)

for i in range(len(files_list_sag)):
    torch.cuda.empty_cache()
    filename = files_list_sag[i]
    x2= inf.get_data_from_filename(filename, sag_dataset)
    x2 = torch.tensor(x2).unsqueeze(0).repeat(32,1,1,1)
    model_sag = model_sag.to('cuda')
    with torch.no_grad():
        model_sag.eval()
        output = model_sag(x2.float().to('cuda'))
    output = output[0].squeeze()
    mask = ((output / output.max()) > 0.05).float().to('cpu').numpy()
    probability = ((output - output.min()) / (output.max() - output.min())).float().to('cpu').numpy()
    print("slice num : ", i)
    base_name, ext = os.path.splitext(files_list_sag[i])
    b_base = os.path.basename(base_name)
    prob_path = os.path.join(prob_out_sag, f"{b_base}.npy")
    mask_path = os.path.join(raw_npy_sag_output, f"{b_base}.npy")
    np.save(prob_path, probability)
    np.save(mask_path, mask)
    print(f'this is base_name: {files_list_sag[i]}')
    display([x2[0].permute([1,2,0]).squeeze(), x2.squeeze(), mask], epoch=os.path.basename(base_name), save_path= None, is_inference=True, inference_path=(raw_png_output+'/sag'))

for i in range(len(files_list_cor)):
    torch.cuda.empty_cache()
    filename = files_list_cor[i]
    x2= inf.get_data_from_filename(filename, cor_dataset)
    x2 = torch.tensor(x2).unsqueeze(0).repeat(32,1,1,1)
    model_cor = model_cor.to('cuda')
    with torch.no_grad():
        model_cor.eval()
        output = model_cor(x2.float().to('cuda'))
    output = output[0].squeeze()
    mask = ((output / output.max()) > 0.05).float().to('cpu').numpy()
    probability = ((output - output.min()) / (output.max() - output.min())).float().to('cpu').numpy()
    print("slice num : ", i)
    base_name, ext = os.path.splitext(files_list_cor[i])
    b_base = os.path.basename(base_name)
    prob_path = os.path.join(prob_out_cor, f"{b_base}.npy")
    mask_path = os.path.join(raw_npy_cor_output, f"{b_base}.npy")
    np.save(prob_path, probability)
    np.save(mask_path, mask)
    print(f'this is base_name: {files_list_cor[i]}')
    display([x2[0].permute([1,2,0]).squeeze(), x2.squeeze(), mask], epoch=os.path.basename(base_name), save_path= None, is_inference=True, inference_path=(raw_png_output+'/cor'))

for i in range(len(files_list_ax)):
    torch.cuda.empty_cache()
    filename = files_list_ax[i]
    x2= inf.get_data_from_filename(filename, ax_dataset)
    x2 = torch.tensor(x2).unsqueeze(0).repeat(32,1,1,1)
    model_ax = model_ax.to('cuda')
    with torch.no_grad():
        model_ax.eval()
        output = model_ax(x2.float().to('cuda'))
    output = output[0].squeeze()
    mask = ((output / output.max()) > 0.05).float().to('cpu').numpy()
    probability = ((output - output.min()) / (output.max() - output.min())).float().to('cpu').numpy()
    print("slice num : ", i)
    base_name, ext = os.path.splitext(files_list_ax[i])
    b_base = os.path.basename(base_name)
    prob_path = os.path.join(prob_out_ax, f"{b_base}.npy")
    mask_path = os.path.join(raw_npy_ax_output, f"{b_base}.npy")
    np.save(prob_path, probability)
    np.save(mask_path, mask)
    print(f'this is base_name: {files_list_ax[i]}')
    display([x2[0].permute([1,2,0]).squeeze(), x2.squeeze(), mask], epoch=os.path.basename(base_name), save_path= None, is_inference=True, inference_path=(raw_png_output+'/ax'))



/home/zimu/anaconda3/lib/python3.11/site-packages/efficientnet_pytorch/utils.py:275: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  x = F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)


slice num :  0
this is base_name: zhao_res/raw_png/sag/Pig_25_slice000.png
converting.........................................
writing to zhao_res/raw_png_output/sag/Pig_25_slice000.png
(128, 256)
slice num :  1
this is base_name: zhao_res/raw_png/sag/Pig_25_slice001.png
converting.........................................
writing to zhao_res/raw_png_output/sag/Pig_25_slice001.png
(128, 256)
slice num :  2
this is base_name: zhao_res/raw_png/sag/Pig_25_slice002.png
converting.........................................
writing to zhao_res/raw_png_output/sag/Pig_25_slice002.png
(128, 256)
slice num :  3
this is base_name: zhao_res/raw_png/sag/Pig_25_slice003.png
converting.........................................
writing to zhao_res/raw_png_output/sag/Pig_25_slice003.png
(128, 256)
slice num :  4
this is base_name: zhao_res/raw_png/sag/Pig_25_slice004.png
converting.........................................
writing to zhao_res/raw_png_output/sag/Pig_25_slice004.png
(128, 256)
slice num :  5


In [60]:

inf.unpad_images('zhao_res/raw_npy_output/sag', 'zhao_res/unpad_npy_output/sag', original_height=112, original_width=256)
inf.unpad_images('zhao_res/raw_npy_output/cor', 'zhao_res/unpad_npy_output/cor', original_height=256, original_width=112)
inf.unpad_images('zhao_res/raw_npy_output/ax', 'zhao_res/unpad_npy_output/ax', original_height=256, original_width=256)
input_dir_sag = 'zhao_res/unpad_npy_output/sag'
input_dir_cor = 'zhao_res/unpad_npy_output/cor'
input_dir_ax = 'zhao_res/unpad_npy_output/ax'
output_dir_sag = volumn_out + '/sag'
output_dir_cor = volumn_out + '/cor'
output_dir_ax = volumn_out + '/ax'
source_dir = images_dir
inf.apply_fslswapdim_to_folder(output_dir_cor, output_dir_cor+'sw')
inf.stack_slices_and_save_nifti(input_dir_sag, output_dir_sag, source_dir, 2)
inf.stack_slices_and_save_nifti(input_dir_cor, output_dir_cor, source_dir, 1)
inf.stack_slices_and_save_nifti(input_dir_ax, output_dir_ax, source_dir, 0)
inf.pply_fslswapdim_to_folder(output_dir_cor, output_dir_cor+'sw')
inf.apply_fslswapdim_to_folder2(output_dir_sag, output_dir_sag+'sw')
inf.run_fslmaths(output_dir_sag+'sw', output_dir_cor+'sw', output_dir_ax, final_out)
inf.apply_fslswapdim_to_folder3(final_out, final_out+'sw')
inf.copy_header_apply(final_out+'sw', images_dir, final_out+'sw_m')

Processed Pig_29_mask.nii.gz
Processed Pig_28_mask.nii.gz
Processed Pig_26_mask.nii.gz
Processed Pig_25_mask.nii.gz
Processed Pig_27_mask.nii.gz
Saved 3D NIfTI volume for Pig 26 to zhao_res/volumn_out/sag/Pig_26_mask.nii.gz
Saved 3D NIfTI volume for Pig 28 to zhao_res/volumn_out/sag/Pig_28_mask.nii.gz
Saved 3D NIfTI volume for Pig 29 to zhao_res/volumn_out/sag/Pig_29_mask.nii.gz
Saved 3D NIfTI volume for Pig 27 to zhao_res/volumn_out/sag/Pig_27_mask.nii.gz
Saved 3D NIfTI volume for Pig 25 to zhao_res/volumn_out/sag/Pig_25_mask.nii.gz
Saved 3D NIfTI volume for Pig 26 to zhao_res/volumn_out/cor/Pig_26_mask.nii.gz
Saved 3D NIfTI volume for Pig 28 to zhao_res/volumn_out/cor/Pig_28_mask.nii.gz
Saved 3D NIfTI volume for Pig 29 to zhao_res/volumn_out/cor/Pig_29_mask.nii.gz
Saved 3D NIfTI volume for Pig 27 to zhao_res/volumn_out/cor/Pig_27_mask.nii.gz
Saved 3D NIfTI volume for Pig 25 to zhao_res/volumn_out/cor/Pig_25_mask.nii.gz
Saved 3D NIfTI volume for Pig 26 to zhao_res/volumn_out/ax/Pig_26


          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!


          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!



Processed and saved: zhao_res/final_out/Pig_26_mask.nii.gz



          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!


          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!



Processed and saved: zhao_res/final_out/Pig_28_mask.nii.gz


...

Processed and saved: zhao_res/final_out/Pig_29_mask.nii.gz



          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!


          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!



Processed and saved: zhao_res/final_out/Pig_27_mask.nii.gz



          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!


          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!



Processed and saved: zhao_res/final_out/Pig_25_mask.nii.gz
Processed Pig_29_mask.nii.gz
Processed Pig_28_mask.nii.gz
Processed Pig_26_mask.nii.gz
Processed Pig_25_mask.nii.gz
Processed Pig_27_mask.nii.gz
